In [14]:
# 1.导入库和模块

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.neural_network import MLPRegressor as MLP
from sklearn.ensemble import AdaBoostRegressor as ABR
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.linear_model import LinearRegression as LR
from sklearn.linear_model import Ridge as R
from sklearn.linear_model import Lasso as L
from math import sqrt

In [11]:
# 2.查看数据

data = pd.read_csv('./dadas/MovieSoure.csv')
data.head()

,导演,编剧,演员,类型,电影时长(分钟),上映时间(年),评分,地区,语言
0,雷德利·斯科特,"大卫·弗兰佐尼,约翰·洛根,威廉姆·尼克尔森","罗素·克劳,华金·菲尼克斯,康妮·尼尔森,奥列佛·里德,理查德·哈里斯,德里克·雅各比,杰曼...",剧情,155,2000,8.5,德国,西班牙语
1,葛伦·盖洛,"菲利普·J·巴特尔,亚伦·布洛卡","利百加·科汉,克里斯·萨尔瓦多,迈克尔E.R.沃克,敏科·斯荳,莱斯利·乔丹,素玛立·蒙塔诺...",喜剧,80,2009,6.6,法国,汉语普通话
2,约翰·R·莱昂耐迪,Michael D. Weiss,"埃里克·里夫利,埃莉卡·杜兰斯,达斯汀·米利甘,吉娜·赫尔顿,林赛·麦克斯维尔",剧情,92,2006,6.1,印度,汉语粤语
3,罗伯特·D·韦布,"莫里斯·杰拉蒂,罗伯特·巴克纳","Richard Egan,Debra Paget,Elvis Presley",剧情,89,1956,7.2,法国,德语
4,卡洛斯·沙尔丹哈,"迈克尔·伯格 ,迈克尔·J·威尔森,彼得·阿克曼","雷·罗马诺,约翰·雷吉扎莫,丹尼斯·利瑞,杰克·布莱克",喜剧,81,2002,8.6,英国,法语


In [12]:
# 3.选取分析数据

df = data.iloc[:,-6:]
df.head()

,类型,电影时长(分钟),上映时间(年),评分,地区,语言
0,剧情,155,2000,8.5,德国,西班牙语
1,喜剧,80,2009,6.6,法国,汉语普通话
2,剧情,92,2006,6.1,印度,汉语粤语
3,剧情,89,1956,7.2,法国,德语
4,喜剧,81,2002,8.6,英国,法语


In [13]:
# 4.建立预测评分函数

def predict_graden(df,movietype,movietime,movieyear,moviegraden,movieplace,movielanguage,model):
    df.loc[len(df)] = [movietype,movietime,movieyear,moviegraden,movieplace,movielanguage]
    
    dftype = pd.get_dummies(df['类型'],prefix='类型')
    datas = pd.concat([df,dftype],axis=1)
    dfplace = pd.get_dummies(df['地区'],prefix='地区')
    datas = pd.concat([datas,dfplace],axis=1)
    dflanguage = pd.get_dummies(df['语言'],prefix='语言')
    datas = pd.concat([datas,dflanguage],axis=1)
    
    datas = datas.loc[:,(datas.columns != '类型') & (datas.columns != '地区') & (datas.columns != '语言')]
    
    x = datas.loc[:,datas.columns != '评分'].values
    y = datas.loc[:,datas.columns == '评分'].values.reshape(-1,1)
    x_train, y_train, x_test, y_test = x[:-1], y[:-1], x[-1], y[-1]
    x_test = x_test.reshape(1,-1)
    
    Model = model()
    Model.fit(x_train,y_train)
    y_pred = Model.predict(x_test)
    
    return y_pred,datas

In [17]:
# 5.建立评分模型评价score函数

def predict_score(datas,model):
    x = datas.loc[:,datas.columns != '评分'].values
    y = datas.loc[:,datas.columns == '评分'].values.reshape(-1,1)
    x_train, x_test, y_train, y_test = model_selection.train_test_split(x,y,test_size=0.3)
    
    Model = model()
    Model.fit(x_train,y_train)
    pre = Model.predict(x_test)
    
    error_sum = 0
    for i,j in zip(y_test,pre):
        error_sum+=sqrt((i-j)**2)
    pre_sum=sum(pre)
    
    return (1-(error_sum/pre_sum))

In [18]:
# 6.打印各模型准确率及预测值

modellist = [MLP,ABR,DTR,RFR,LR,R,L]
for m in modellist:

    # movietype = input()
    # movietime = eval(input())
    # movieyear = eval(input())
    # moviegraden = eval(input())
    # movieplace = input()
    # movielanguage = input()
    # model = input()

    movietype = '喜剧'
    movietime = 80
    movieyear = 2009
    moviegraden = 6.6
    movieplace = '法国'
    movielanguage = '汉语普通话'
    model = m

    garden,datas = predict_graden(df,movietype,movietime,movieyear,moviegraden,movieplace,movielanguage,model)
    score = predict_score(datas,model)
    
    print(str(model)+'  '+str(score))
    
    garden_train = moviegraden
    garden_test = "{:.2}".format(garden.flatten()[0])
    print('期望评分' + ' ' + str(garden_train))
    print('预测评分' + ' ' + str(garden_test))

D:\ANACONDA\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
D:\ANACONDA\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


<class 'sklearn.neural_network._multilayer_perceptron.MLPRegressor'>  0.8772255567713343
期望评分 6.6
预测评分 6.8


D:\ANACONDA\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
D:\ANACONDA\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


<class 'sklearn.ensemble._weight_boosting.AdaBoostRegressor'>  0.8648272890643154
期望评分 6.6
预测评分 6.7
<class 'sklearn.tree._classes.DecisionTreeRegressor'>  0.8519367399377529
期望评分 6.6
预测评分 6.6


D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


<class 'sklearn.ensemble._forest.RandomForestRegressor'>  0.886350564245825
期望评分 6.6
预测评分 6.6
<class 'sklearn.linear_model._base.LinearRegression'>  [0.88153593]
期望评分 6.6
预测评分 7.1
<class 'sklearn.linear_model._ridge.Ridge'>  [0.88393468]
期望评分 6.6
预测评分 7.1
<class 'sklearn.linear_model._coordinate_descent.Lasso'>  0.875860964781132
期望评分 6.6
预测评分 7.2
